In [3]:
from bs4 import BeautifulSoup
import os

from aeneas.executetask import ExecuteTask
from aeneas.task import Task

root = "../../workspace/SaveTheCat_epub/"

# Folder containing the XHTML files
infolder_path = root + "plaintext"
outfolder_path = root + "sync_text"

# Initialize ID counter
id_counter = 1

# Iterate through each file in the folder
for filename in sorted(os.listdir(infolder_path), key = lambda x: x[:4]):
    if filename.endswith('.xhtml'):
        infile_path = os.path.join(infolder_path, filename)
        outfile_path = os.path.join(outfolder_path, filename)

        # Read the XHTML file
        with open(infile_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the XHTML content using BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Unwrap all <span class="RunIn"> elements
        for spanx in soup.find_all('span', class_='RunIn'):
            spanx.unwrap()

        # Save the modified content back to the file
        with open(outfile_path, 'w', encoding='utf-8') as file:
                file.write(str(soup))
                
# Folder containing the XHTML files
infolder_path = root + "sync_text"
outfolder_path = root + "sync_text"

# Initialize ID counter
id_counter = 1

# Iterate through each file in the folder
for filename in sorted(os.listdir(infolder_path), key = lambda x: x[:4]):
    if filename.endswith('.xhtml'):
        infile_path = os.path.join(infolder_path, filename)
        outfile_path = os.path.join(outfolder_path, filename)

        # Read the XHTML file
        with open(infile_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the XHTML content using BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Unwrap all <span class="RunIn"> elements
        for spanx in soup.find_all('span', class_='RunIn'):
            spanx.unwrap()

        # Find the <body> element
        body_element = soup.find('body')

        # Find all text elements within the <body> element
        text_elements = body_element.find_all(string=True)

        # Iterate through the text elements and wrap them with span tags and unique IDs
        for element in text_elements:
    
            if '.' in element:  # Check if there's a dot in the text

                parts = element.split('.')
                new_parts = []
                for i, part in enumerate(parts):
                    if part.strip():
                        span_tag = soup.new_tag('span')
                        span_tag['id'] = f'f{str(id_counter).zfill(6)}'
                        span_tag.string = part.strip() 

                        if i < len(parts) -1:
                            if parts[i+1].strip():
                                if parts[i+1][0] == "”":
                                    span_tag.string += '.” '
                                    parts[i+1] = parts[i+1][1:]
                                else:
                                    span_tag.string += '. '
                            else:
                                    span_tag.string += '. '

                        else: 
                            if span_tag.string[-1] != "“":
                                span_tag.string += ' '

                        new_parts.append(span_tag)
                        id_counter += 1
                    else:
                        new_parts.append(part)
                element.replace_with(*new_parts)
            else:
                if element.strip():
                    span_tag = soup.new_tag('span')
                    span_tag['id'] = f'f{str(id_counter).zfill(6)}'
                    element.wrap(span_tag)
                    id_counter += 1

        # Save the modified content back to the file
        with open(outfile_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))
            

# audiofiles = sorted(os.listdir(root + "audio"), key = lambda x: x[:2])
# synctextfiles = sorted(os.listdir(root + "sync_text"), key = lambda x: x[:4])

audiofiles = sorted(os.listdir(root + "audio"), key = lambda x: x[:2])[:1]

synctextfiles = sorted(os.listdir(root + "sync_text"), key = lambda x: x[:4])[:1]

assert len(audiofiles) == len(synctextfiles)

print(*zip(audiofiles, synctextfiles ))

for i in range(len(audiofiles)):
    print(i)
    # create Task object
    config_string = u"task_language=eng|os_task_file_format=smil|os_task_file_smil_audio_ref=../audio/{0}|os_task_file_smil_page_ref=../text/{1}|is_text_type=unparsed|is_text_unparsed_id_regex=f[0-9]+|is_text_unparsed_id_sort=numeric".format(audiofiles[i], synctextfiles[i])
    task = Task(config_string=config_string)
    task.audio_file_path_absolute =  root + u"audio/" + audiofiles[i]
    task.text_file_path_absolute = root + u"sync_text/" + synctextfiles[i]
    task.sync_map_file_path_absolute = root + "/smil/{}.smil".format(synctextfiles[i])

    # process Task
    Execute = ExecuteTask(task)
    Execute.logger.tee = True
    Execute.execute()

    # output sync map to file
    task.output_sync_map_file()

('02. 002.mp3', 'c00.xhtml')
0
Create Task
Executeing task
[DEBUG] 2024-05-09 18:12:26.284072 ExecuteTask: Executing task...
[DEBUG] 2024-05-09 18:12:26.284389 ExecuteTask: Both audio and text input file are present
[DEBUG] 2024-05-09 18:12:26.284417 ExecuteTask: Executing single level task...
[DEBUG] 2024-05-09 18:12:26.284429 ExecuteTask: STEP 1 BEGIN (extract MFCC real wave)
[DEBUG] 2024-05-09 18:12:26.284456 AudioFileMFCC: Initializing MFCCs...
[DEBUG] 2024-05-09 18:12:26.284477 AudioFile: Loading audio data...
[DEBUG] 2024-05-09 18:12:26.284849 AudioFile: self.file_format is None or not good => converting self.file_path
[DEBUG] 2024-05-09 18:12:26.285123 AudioFile: Temporary PCM16 mono WAVE file: '/tmp/tmp4o9d8gkf.wav'
[DEBUG] 2024-05-09 18:12:26.285152 AudioFile: Converting audio file to mono...
[DEBUG] 2024-05-09 18:12:26.285244 FFMPEGWrapper: Calling with arguments '['ffmpeg', '-i', '../../workspace/SaveTheCat_epub/audio/02. 002.mp3', '-ac', '1', '-ar', '16000', '-y', '-map_met

In [4]:
import aeneas
import numpy as np

m1 = np.loadtxt('m1.txt')
m2 = np.loadtxt('m2.txt')
delta = 3000
# print(m1)

result_0 = aeneas.cdtw.cdtw.compute_best_path(m1, m1, delta)
result_0

Time elapsed in compute_best_path: 0.950420 seconds


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (1, 16),
 (2, 16),
 (3, 16),
 (4, 16),
 (5, 16),
 (6, 16),
 (7, 16),
 (8, 16),
 (9, 16),
 (10, 16),
 (11, 16),
 (12, 16),
 (13, 16),
 (14, 16),
 (15, 16),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (25, 25),
 (26, 26),
 (27, 27),
 (28, 28),
 (29, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (37, 38),
 (37, 39),
 (37, 40),
 (37, 41),
 (37, 42),
 (37, 43),
 (37, 44),
 (37, 45),
 (37, 46),
 (37, 47),
 (37, 48),
 (37, 49),
 (37, 50),
 (37, 51),
 (37, 52),
 (37, 53),
 (37, 54),
 (37, 55),
 (37, 56),
 (37, 57),
 (37, 58),
 (37, 59),
 (37, 60),
 (37, 61),
 (37, 62),
 (37, 63),
 (37, 64),
 (37, 65),
 (37, 66),
 (37, 67),
 (37, 68),
 (37, 69),
 (37, 70),
 (37, 71),
 (37, 72),
 (37, 73),
 (37, 74),
 (37, 75),
 (37, 76),
 (37, 77),
 (

In [2]:
import aeneas_rust
import numpy as np

m1 = np.loadtxt('m1.txt')
m2 = np.loadtxt('m2.txt')
delta = 3000

# result = rust_extension.compute_best_path([[0.1, 0.2], [0.3, 0.4]], [[0.1, 0.2], [0.3, 0.4]], 1)
result = aeneas_rust.compute_best_path(m1, m1, delta)
result

Time 1: 2.48557ms
Time 2: 24.186µs
Time 3: 1.025592122s
Time 4: 512.884172ms
Time 5: 967.428µs
Time 6: 30ns


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (1, 16),
 (2, 16),
 (3, 16),
 (4, 16),
 (5, 16),
 (6, 16),
 (7, 16),
 (8, 16),
 (9, 16),
 (10, 16),
 (11, 16),
 (12, 16),
 (13, 16),
 (14, 16),
 (15, 16),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (25, 25),
 (26, 26),
 (27, 27),
 (28, 28),
 (29, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (37, 38),
 (37, 39),
 (37, 40),
 (37, 41),
 (37, 42),
 (37, 43),
 (37, 44),
 (37, 45),
 (37, 46),
 (37, 47),
 (37, 48),
 (37, 49),
 (37, 50),
 (37, 51),
 (37, 52),
 (37, 53),
 (37, 54),
 (37, 55),
 (37, 56),
 (37, 57),
 (37, 58),
 (37, 59),
 (37, 60),
 (37, 61),
 (37, 62),
 (37, 63),
 (37, 64),
 (37, 65),
 (37, 66),
 (37, 67),
 (37, 68),
 (37, 69),
 (37, 70),
 (37, 71),
 (37, 72),
 (37, 73),
 (37, 74),
 (37, 75),
 (37, 76),
 (37, 77),
 (

In [5]:
result[:30000] == result_0[:30000]

True

In [7]:
len(result), len(result_0)

(36661, 35162)